## Description 
This example demonstrates how to run GeniERuntime through OneWorkflow. A small dummy script for demonstrating SifIO is also provided. This example uses a parametrized containership model from [GeniE Snack Pack](https://sesam.dnv.com/genie_utils/hullforms/containership.html). *AP* and *FP* are parametrized and input given from a Pandas dataframe as shown below.




In [ ]:
from pathlib import Path
from dnv.oneworkflow.utils.workunit_extension import *
from dnv.oneworkflow.utils.starter import *
from dnv.oneworkflow import OneWorkflowClient
from pathlib import Path
import os
oneWorkflowTMPFolder = r'D:\OneWorkflowTmp' #due to possible issues with long file paths we prefer to have this folder at the root
if not os.path.exists(oneWorkflowTMPFolder):
    try:
        print("Trying to create tmp folder for one workflow local execution")
        os.mkdir(oneWorkflowTMPFolder)
    except:
        print("did not manage to create tmp folder for local execution. Check that you have privileges to create it or try to manually create it from the coomand line.")

workspaceId = "GeniERuntimeExample"
# local workspace, all results will be put here after local or cloud runs
# location of common files for all analysis, has to be below workspacePath and in the folder names CommonFilesr
root_folder = os.getcwd()
print(root_folder)
workspacePath = str(Path(root_folder, 'Workspace'))
cloudRun = False
#If running locally the code below will also start the local workflow host.
workflow_client = one_workflow_client(workspace_id = workspaceId, workspace_path = workspacePath, cloud_run = cloudRun,
                                      local_workflow_runtime_temp_folder_path = oneWorkflowTMPFolder, platform=Platform.WINDOWS, max_cores=5,auto_deploy_option = AutoDeployOption.DEV)
workflow_client.start_workflow_runtime_service()
if (cloudRun):
    workflow_client.login()
if not workflow_client.upload_common_files(FileOptions(max_size="524MB",patterns=["**/*.*"], overwrite=True)):
    print("Upload failed")

In [ ]:
import pandas as pd
from SesamHelpers import *
import shutil
import json
# we must delete existing results locally before generating new results
local__result_path = Path(workspacePath, workflow_client.results_directory)
if os.path.isdir(local__result_path):
    shutil.rmtree(local__result_path)

#parametrized values
df = pd.DataFrame({'AP':  ["0m", "0.5m", "1m"], 'FP': ["150m", "250m", "500m"]})
workUnit = GeniERuntimeTaskCreator("ContainerHull_template.js", df,workflow_client).get_genieruntime_work_unit(cloudRun, workspacePath)
downloadOptions = FileOptions(
    max_size="1112MB",
    patterns=["**/T1.FEM", "**/*.csv"])
job = workflow_client.create_job(workUnit)

#for debugging only
#job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
await run_workflow_async(job, workflow_client, downloadOptions)


### CLose client -must be done before a new job can be started in a different notebook
Will remove all job and blob folders.

In [ ]:
workflow_client.local_workflow_runtime_service.stop_service()

## Postprocessing
The code below prints out node and element counts for the different models and present the results in a table.


In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import glob

lc = 11
frames = []
for folder in glob.glob(f"{local__result_path}\\*"):
    data = pd.read_csv(Path(folder, "nodeCount.csv"), index_col=0)
    data.index = [folder.split('\\')[-1]]
    frames.append(data)
df = pd.concat(frames)
df=df.rename_axis("Loadcase")
display(df)
